In [1]:
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
import pickle
import random
from seqeval.metrics import f1_score

torch.manual_seed(1)

In [2]:
path_train = '/home/truongnd/Desktop/hashing_embeddings/data/eng.train'
path_dev = '/home/truongnd/Desktop/hashing_embeddings/data/eng.testa'
path_test = '/home/truongnd/Desktop/hashing_embeddings/data/eng.testb'

In [3]:
def prepare_dataset(path_train):
    with open(path_train, 'r') as f:
        train = f.readlines()
    
    sentences = []
    sentence = []
    for i in train:
        word = i.split()
        if len(word) > 1:
            l = (word[0], word[-1].replace('B-','I-'))
            sentence.append(l)
        if len(word) == 0 and len(sentence) > 0:
            sentences.append(sentence)
            sentence = []

    dataset = []
    for i in sentences:
        sent = [t[0] for t in i]
        tag = [t[-1] for t in i]
        dataset.append((sent, tag))
    
    return dataset

In [4]:
train_set = prepare_dataset(path_train)
dev_set = prepare_dataset(path_dev)
test_set = prepare_dataset(path_test)

In [5]:
def prepare_sequence(seq, to_ix):
    idxs = []
    for w in seq:
        if w.lower() in to_ix:
            idxs.append(to_ix[w.lower()])
        else:
            idxs.append(to_ix['<unk>'])
    return torch.tensor(idxs, dtype=torch.long)

def prepare_tag_sequence(seq, tag_to_ix):
    idxs = [tag_to_ix[i] for i in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [6]:
word_to_ix = {}
tag_to_ix = {}
for sent, tags in train_set:
    for word in sent:
        if word.lower() not in word_to_ix:
            word_to_ix[word.lower()] = len(word_to_ix)

# Add <unk> token
word_to_ix['<unk>'] = len(word_to_ix)

START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
tag_to_ix = {'O': 0, 'I-LOC': 1, 'I-PER': 2, 'I-ORG': 3, 'I-MISC': 4, START_TAG: 5, STOP_TAG: 6}
ix_to_tag = {0: 'O', 1: 'I-LOC', 2: 'I-PER', 3: 'I-ORG', 4: 'I-MISC', 5: START_TAG, 6: STOP_TAG}

In [7]:
def argmax(vec):
    _, idx = torch.max(vec, 1)
    return idx.item()


def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

In [8]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))
    
    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def _get_lstm_features(self, sentence):
        #self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, _ = self.lstm(embeds)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score
    
    def _viterbi_decode(self, feats):
        backpointers = []

        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path
    
    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [9]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BiLSTM_CRF(
  (word_embeds): Embedding(21011, 300)
  (lstm): LSTM(300, 128, bidirectional=True)
  (hidden2tag): Linear(in_features=256, out_features=7, bias=True)
)

In [10]:
def check_f1_valid(dev_set):
    lts = []
    lts_pred = []
    for sent, label in dev_set:
        with torch.no_grad():
            lts.append(label)
            
            sent_in = prepare_sequence(sent, word_to_ix).to(device)
            #print(sent_in)
            _, label = model(sent_in)
            lts_pred.append([ix_to_tag[i] for i in label])
    print(lts)
    print(lts_pred)
                                    
    return f1_score(lts, lts_pred)

In [ ]:
EPOCH = 500
train_loss = 0.
f1_dev = 0

for epoch in tqdm(range(EPOCH)):
    N_train = len(train_set)
    sum_train_loss = 0
    random.shuffle(train_set)
    
    for sentence, tags in train_set:
        model.zero_grad()
        sent_in = prepare_sequence(sentence, word_to_ix).to(device)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sent_in, targets)
        
        loss.backward()
        optimizer.step()
        sum_train_loss += loss.item()
        
    f1_dev_per_epoch = check_f1_valid(dev_set)
    
    if epoch == 0:
        train_loss = sum_train_loss/N_train
        f1_dev = f1_dev_per_epoch
        
    print('Epoch {}: train_loss is {} \t f1_dev is {}'.format(epoch, sum_train_loss/N_train, f1_dev_per_epoch))
    
    if f1_dev_per_epoch > f1_dev:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'f1_dev': f1_dev_per_epoch
        }, './best_model_bilstm_crf.tar')
        f1_dev = f1_dev_per_epoch
        
    if epoch == EPOCH - 1:
        print('Done training of model!')

  0%|          | 0/500 [00:00<?, ?it/s]

In [ ]:
lts = []
lts_pred = []

with torch.no_grad():
    for sent, label in test_set:
        lts.append(label)
        sent_in = prepare_sequence(sent, word_to_ix).to(device)
        _, label = model(sent_in)
        lts_pred.append([ix_to_tag[i] for i in label])
        
f1_score(lts, lts_pred)

In [ ]:
# Load best model
state = torch.load('./best_model_bilstm_crf.tar')
model_new = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)

model_new.load_state_dict(state['model_state_dict'])
model_new.to(device)

lts = []
lts_pred = []

with torch.no_grad():
    for sent, label in test_set:
        lts.append(label)
        sent_in = prepare_sequence(sent, word_to_ix).to(device)
        , label = model_new(sent_in)
        lts_pred.append([ix_to_tag[i] for i in label])
        
f1_score(lts, lts_pred)